In [1]:
import os
import numpy as np
import h5py
import torch
import cv2
import glob
import torch.utils.data as udata

In [2]:
class Kernels:
    
    @staticmethod
    def kernel_2d(
        kernel_size, 
        sigma
    ):
        center = (kernel_size - 1) / 2
        kernel = np.fromfunction(
            lambda x, y: 
                np.exp( -0.5 * ((x - center) ** 2 + (y - center) ** 2) / (sigma ** 2)), 
            (kernel_size, kernel_size), 
            dtype=float
        )
        return kernel

In [3]:
def normalize(data):
    return data / 255.

def Im2Patch(img, win, stride=1):
    k = 0
    endc = img.shape[0]
    endw = img.shape[1]
    endh = img.shape[2]

    patch = img[:, 0:endw-win+0+1:stride, 0:endh-win+0+1:stride]
    TotalPatNum = patch.shape[1] * patch.shape[2]
    Y = np.zeros([endc, win*win,TotalPatNum], np.float32)
    for i in range(win):
        for j in range(win):

            patch = img[:,i:endw-win+i+1:stride,j:endh-win+j+1:stride]
            Y[:,k,:] = np.array(patch[:]).reshape(endc, TotalPatNum)
            k = k + 1
    return Y.reshape([endc, win, win, TotalPatNum])

def prepare_data(data_path='./data', patch_size=32, stride=16, gksize=11, gsigma=3):
    print('processing data')
    
    kernel = Kernels.kernel_2d(gksize, gsigma)

    files = glob.glob(os.path.join(data_path, 'BSD400', '*.png'))
    files.sort()
    train_file_name = 'datasets/train_ps%d_stride%d.h5' % (patch_size, stride)

    h5f = h5py.File(train_file_name, 'w')
    train_num = 0
    for i in range(len(files)):
        img = cv2.imread(files[i])
        
        Img = cv2.filter2D(np.float32(img), -1, kernel, borderType=cv2.BORDER_CONSTANT)
        h, w, c = Img.shape
        
        Img = np.expand_dims(Img[:,:,0].copy(), 0)
        Img = np.float32(normalize(Img))

        patches = Im2Patch(Img, win=patch_size, stride=stride)
        print("file: %s # samples: %d" % (files[i], patches.shape[3]), end='\r')
        for n in range(patches.shape[3]):
            data = patches[:,:,:,n].copy()
            h5f.create_dataset(str(train_num), data=data)
            train_num += 1
    h5f.close()

    print('\ntraining set, # samples %d\n' % train_num)

In [172]:
prepare_data('./data', 64, 32)

processing data
file: ./data\BSD400\test_400.png # samples: 16
training set, # samples 6400



In [4]:
class Dataset(udata.Dataset):
    def __init__(self, train=True, patch_size=32, stride=16, size='full', seed=1234):
        super(Dataset, self).__init__()
        self.train = train

        self.train_file_name = 'datasets/train_ps%d_stride%d.h5' % (patch_size, stride)
        h5f = h5py.File(self.train_file_name, 'r')

        self.all_keys = sorted(list(h5f.keys()))
        np.random.seed(seed)
        np.random.shuffle(self.all_keys)
        
        self.size = len(self.all_keys) if size == 'full' else size
        self.keys = self.all_keys[:self.size]
        h5f.close()
    
    def set_size(self, size='full'):
        self.size = len(self.all_keys) if size == 'full' else size
        self.keys = self.all_keys[:self.size]
    
    def is_full(self):
        return self.size == len(self.all_keys)

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, index):
        h5f = h5py.File(self.train_file_name, 'r')
        key = self.keys[index]
        data = np.array(h5f[key])
        h5f.close()
        return torch.Tensor(data)

In [11]:
DS = Dataset(seed=1234, size='full')

# MODELS

In [5]:
import torch
import torch.nn as nn
import os

In [6]:
class CNN_Model(nn.Module):
    def __init__(self, num_of_layers=10, kernel_size=3, padding=1, features=64, gksize=11, gsigma=3):
        super(CNN_Model, self).__init__()
        # We are only interested in grayscale
        channels = 1
        gkernel = Kernels.kernel_2d(gksize, gsigma)
        
        def Conv2d_train(in_channels, out_channels):
            return nn.Conv2d(
                in_channels  = in_channels, 
                out_channels = out_channels, 
                kernel_size  = kernel_size, 
                padding      = padding, 
                bias         = False
            )
        
        def Conv2d_blur(kernel):
            layer = nn.Conv2d(
                in_channels  = channels, 
                out_channels = channels, 
                kernel_size  = kernel.shape, 
                padding      = 0, 
                bias         = False
            )
            
            w, h = kernel.shape
            gaussian_kernel = torch.ones(1, 1, w, h)
            gaussian_kernel[0, 0] = torch.from_numpy(kernel)
            layer.weight = torch.nn.Parameter(gaussian_kernel)
            
            return layer
        
        layers = []
        layers.append(Conv2d_train(channels, features))
        layers.append(nn.ReLU(inplace=True))
        
        for _ in range(num_of_layers-2):
            layers.append(Conv2d_train(features, features))
            layers.append(nn.BatchNorm2d(features))
            layers.append(nn.ReLU(inplace=True))
        
        layers.append(Conv2d_train(features, channels))
        
        layers.append(Conv2d_blur(gkernel))
        
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        out = self.network(x)
        return out

In [7]:
model = CNN_Model(gksize=7)

In [10]:
model.network

Sequential(
  (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): ReLU(inplace=True)
  (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU(inplace=True)
  (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU(inplace=True)
  (11): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (13): ReLU(inplace=True)
  (14): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=Fal

In [155]:
model.network[:-1].parameters()

<generator object Module.parameters at 0x000002BEB85F6B10>

# Train

In [160]:
import os
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader

from models import CNN_Model
from dataset import prepare_data, Dataset

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

parser = argparse.ArgumentParser(description="Deblurring")

parser.add_argument("--preprocess",          type=bool,  default=False, help='run prepare_data or not')
parser.add_argument("--patch_size",          type=int,   default=64,    help='dataset patch size')
parser.add_argument("--stride",              type=int,   default=32,    help='dataset stride')
parser.add_argument("--gksize",              type=int,   default=11,    help='blur kernel size')
parser.add_argument("--gsigma",              type=int,   default=3,     help='blur kernel sigma')
parser.add_argument("--dataset_size",        type=int,   default=16,    help='train dataset size to use')
parser.add_argument("--dataset_seed",        type=int,   default=1234,  help='seed to determine dataset order')
parser.add_argument("--network_kernel_size", type=int,   default=3,     help='network convolution kernel size')
parser.add_argument("--network_features",    type=int,   default=64,    help='network numbre of features')
parser.add_argument("--batch_size",          type=int,   default=100,   help="Training batch size")
parser.add_argument("--num_of_layers",       type=int,   default=10,    help="Number of total layers")
parser.add_argument("--epochs",              type=int,   default=50,    help="Number of training epochs")
parser.add_argument("--milestone",           type=int,   default=30,    help="When to decay learning rate; should be less than epochs")
parser.add_argument("--lr",                  type=float, default=1e-3,  help="Initial learning rate")

opt = parser.parse_args()


def main():
    # Load dataset
    print('Loading dataset ...\n')
    
    dataset_train = Dataset(
        patch_size = opt.patch_size, 
        stride     = opt.stride, 
        size       = opt.dataset_size, 
        seed       = opt.dataset_seed
    )

    loader_train = DataLoader(dataset=dataset_train, num_workers=4, batch_size=opt.batch_size, shuffle=True)
    print("# of training samples: %d\n" % int(len(dataset_train)))
    
    print('** Creating network: **')
    net = CNN_Model(
        num_of_layers = opt.num_of_layers, 
        kernel_size   = opt.network_kernel_size, 
        features      = opt.network_features, 
        gksize        = opt.gksize, 
        gsigma        = opt.gsigma
    )
    
    # Loss
    criterion = nn.MSELoss(size_average=False)
    
    # Move to GPU
    model = nn.DataParallel(net).cuda()
    criterion.cuda()
    
    # Optimizer
    # TODO: check if that's correct and doesn't train the fixed convolution layer
    optimizer = optim.Adam(model.network[:-1].parameters(), lr=opt.lr)

    train_loss_log = np.zeros(opt.epochs)
    
    for epoch in range(opt.epochs):
        if epoch < opt.milestone:
            current_lr = opt.lr
        else:
            current_lr = opt.lr / (10.)
        # Learning rate
        for param_group in optimizer.param_groups:
            param_group["lr"] = current_lr
        print('\nlearning rate %f' % current_lr)
        
        # Train
        for i, data in enumerate(loader_train, 0):
            # Training
            model.train()
            model.zero_grad()
            optimizer.zero_grad()

            # Training step
            img_train = Variable(data.cuda())
            
            out_train = model(img_train)
            loss = criterion(out_train, img_train) / (img_train.size()[0]*2)
            
            loss.backward()
            optimizer.step()
            
            train_loss_log[epoch] += loss.item()

        train_loss_log[epoch] = train_loss_log[epoch] / len(loader_train)

        print('Epoch %d: loss=%.4f' %(epoch, train_loss_log[epoch]))
        
        # TODO Remove when sure last layer isn't trained
        print(model.network[-1].weight)
        
        model_name = 'DSseed%d_ps%_stride%s_lr%d_layers%d_kernel%d_features%d' % (
            opt.dataset_seed, 
            opt.patch_size, 
            opt.stride, 
            opt.lr, 
            opt.num_of_layers, 
            opt.network_kernel_size, 
            opt.network_features
        )
        
        model_name_with_size = os.path.join(model_name, 'DSsize%d' % opt.dataset_size)

        model_dir = os.path.join('saved_models', model_name_with_size)
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        torch.save(model.state_dict(), os.path.join(model_dir, 'net_%d.pth' % (epoch)) )


if __name__ == "__main__":
    if opt.preprocess:
        prepare_data(
            patch_size = opt.patch_size, 
            stride     = opt.stride, 
            gksize     = opt.gksize, 
            gsigma     = opt.gsigma
        )
    
    main()

# Test

In [182]:
# TODO maybe compute dataset nb base images like this from patch size and stride
ps = 64
stride = 32
((180 - ps) // stride + 1) ** 2

16

In [171]:
import cv2
import os
import argparse
import glob
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from models import CNN_Model
from util import Kernels

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

parser = argparse.ArgumentParser(description="DeblurringTest")

parser.add_argument("--model_seed",        type=int,   default=1234, help='seed used when training the model')
parser.add_argument("--model_patch_size",  type=int,   default=64,   help='train dataset patch size')
parser.add_argument("--model_stride",      type=int,   default=32,   help='train dataset stride')
parser.add_argument("--model_lr",          type=float, default=1e-3, help='lr used for training the model')
parser.add_argument("--model_num_layers",  type=int,   default=10,   help='number of layers in the model')
parser.add_arugment("--model_kernel_size", type=int,   default=3,    help='kernel size in the model')
parser.add_argument("--model_features",    type=int,   default=64,   help='number of features in the model')
parser.add_argument("--gksize",            type=int,   default=11,   help='blur kernel size')
parser.add_argument("--gsigma",            type=int,   default=3,    help='blur kernel sigma')

opt = parser.parse_args()


def normalize(data):
    return data/255.


def inference(test_data, model):
    files_source = glob.glob(os.path.join(test_data, 'BSD68', '*.png'))

    files_source.sort()
    kernel = Kernels.kernel_2d(opt.gksize, opt.gsigma)
    psnr_results = []
    
    for f in files_source:
        Img = cv2.imread(f)
        Img = cv2.filter2D(np.float32(Img), -1, kernel, borderType=cv2.BORDER_CONSTANT)
        Img = normalize(np.float32(Img[:,:,0]))
        Img = np.expand_dims(Img, 0)
        Img = np.expand_dims(Img, 1)
        ISource = torch.Tensor(Img)
        ISource = Variable(ISource.cuda())
        
        with torch.no_grad():
            IOut = model(ISource)
            psnr_results.append(batch_PSNR(IOut, ISource, 1.))

    return np.mean(psnr_results)


def main():
    
    base_to_patch = ((180 - opt.model_patch_size) // opt.model_stride + 1) ** 2
    
    model_name = 'DSseed%d_ps%_stride%s_lr%d_layers%d_kernel%d_features%d' % (
        opt.model_seed, 
        opt.model_patch_size, 
        opt.model_stride, 
        opt.model_lr, 
        opt.model_num_layers, 
        opt.model_kernel_size, 
        opt.model_features
    )
    
    log_dir = os.path.join('logs', model_name)
    model_dir = os.path.join('saved_models', model_name)
    
    model_DSsizes = os.listdir(model_dir)
    
    results = np.zeros((len(model_DSsizes), 2))
    
    for model_DSsize in model_DSsizes:
        DSsize = int(model_DSsize.split('size')[-1])
        size_idx = (DSsize // base_to_patch) - 1
        
        epochs_dir = os.path.join(model_dir, model_DSsize)
        model_epochs = os.listdir(epochs_dir)
        
        results_epoch = np.zeros(len(model_epochs))
        
        for model_epoch in model_epochs:
            epoch = int(model_epoch.split('_')[-1].split('.')[0]) # + 1
    
            print('Testing with model %s with DSsize %d at epoch %d' % (DSsize, epoch))
            
            net = CNN_Model(
                num_of_features = opt.model_features, 
                kernel_size     = opt.model_kernel_size, 
                features        = opt.model_features, 
                gksize          = opt.gksize, 
                gsigma          = opt.gsigma
            )
            
            model = nn.DataParallel(net).cuda()
            model.load_state_dict(torch.load(os.path.join(epochs_dir, model_epoch)))
            model.eval()
            
            mean_psnr = inference('data', model)
            
            results_epoch[epoch] = mean_psnr
        
        max_epoch = np.argmax(results_epoch)
        max_psnr  = np.max(results_epoch)
        
        results[size_idx, 0] = max_epoch
        results[size_idx, 1] = max_psnr


    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    np.save(os.path.join(log_dir, 'results', results)

    print('Results saved inside Logs!')


if __name__ == "__main__":
    main()